In [1]:
# import statements

import xarray as xr
import dask.array as da
import xesmf as xe
import numpy as np
import matplotlib.pyplot as plt

In [3]:
models = ['ACCESS1-0','ACCESS1-3','CNRM-CM5','IPSL-CM5B-LR', 'GFDL-ESM2G',
         'MIROC-ESM', 'FGOALS-g2','bcc-csm1-1','BNU-ESM','CanESM2','CCSM4',
          'CSIRO-Mk3-6-0','FGOALS-s2','GFDL-CM3','inmcm4',
         'IPSL-CM5A-LR','MIROC5','MPI-ESM-LR','MPI-ESM-P','MRI-CGCM3','NorESM1-M']

# start by reading in the air temperature kernel
ta_kern = xr.open_dataset('/dx05/tylerj/d10/Arctic_Research/CMIP5_Arctic_Amplification/\
CAM5_kernels/t.kernel.plev.nc',use_cftime=True)
lats = ta_kern.lat
lons = ta_kern.lon
# some models have higher vertical resolution than the kernels. We only need 17 levels
plevs = [float(p) for p in ta_kern.plev]

# create a dataset with lat/lons for regridding
ds_out = xr.Dataset({'lat': (['lat'], lats),
                     'lon': (['lon'], lons),
                    }
                   )

for mod in models:
    print(mod)
    # read in first 150 years of surface pressure file
    ps = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/abrupt4xCO2/ps_Amon_'+mod+'_*.nc',
                          parallel=True,
                           combine='by_coords',use_cftime=True).ps.isel(time=slice(None,1800))

    # produce a regridder to map input to CAM5 horizontal grid
    regridder = xe.Regridder(ps,ds_out,'bilinear',periodic=True,reuse_weights=True)

    # regrid the surface pressure, chunk it
    ps = regridder(ps.compute())
    lat = ps.lat
    lon = ps.lon

    # we require the pressure level axis from a vertically varying variable, such as air temp
    ta = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/abrupt4xCO2/ta_Amon_'+mod+'_*.nc',
                          parallel=True,combine='by_coords',
                          use_cftime=True).ta.isel(time=0).sel(plev=plevs)
    plev_1D = ta.plev
    plev_da,foo = xr.broadcast(plev_1D,ps)

    # make bounds array to compute thickness later
    bounds_array = [96250., 88750., 77500., 65000., 55000., 45000., 35000., 27500.,
                        22500., 17500., 12500.,  8500.,  6000.,  4000.,  2500.,  1500.,0]
    bounds_da,foo = xr.broadcast(xr.DataArray(bounds_array,coords={'plev':plev_da.plev},dims='plev'),ps)

    ps = np.array(ps)
    dz = np.zeros_like(plev_da)

    # let's specify some dimensions:
    # ps is 3D (time,lat,lon)
    # we will be iterating through the vertical direction

    for i in range(17):
        # here is where we have to be very space conscious
        # dask does not allow item assignment as it is difficult to parallelize
        plev = np.array(plev_da.isel(plev=i))
        if(i==0):
            # grab bounds and plev, make arrays
            bounds = np.array(bounds_da.isel(plev=i))
            # set the lower layer thickness to be the surface pressure minus the lower bound
            dz[i,:,:,:] = ps[:,:,:] - bounds[:,:,:]
            # if the surface pressure is less than lowest pressure level, set to 0
            dz[i,plev[:,:,:] > ps[:,:,:]] = 0
        else:
            bounds = np.array(bounds_da.isel(plev=slice(i-1,i+1)))
            # where the lower bound is below the surface,
            # set thickness to be difference between upper bound and surface pressure
            # otherwise, do nothing
            dz[i,:,:,:] = np.where(bounds[0,:,:,:] > ps[:,:,:],
                                   ps[:,:,:] - bounds[1,:,:,:],
                                   dz[i,:,:,:])
            # where the center pressure is below the surface, set thickness to 0
            dz[i,plev[:,:,:] > ps[:,:,:]] = 0

    # convert dz to dataarray, save it out
    dz_da = xr.DataArray(dz,coords={'plev':plev_da.plev,'time':plev_da.time,'lat':plev_da.lat,
                                    'lon':plev_da.lon},
                             dims=['plev','time','lat','lon']).rename('dz').transpose('time',
                                                                                      'plev','lat','lon')

    del dz
    
    dz_da.to_netcdf('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/' + mod + '_dz_strat_4xCO2.nc')

ACCESS1-0
Reuse existing file: bilinear_145x192_192x288_peri.nc
ACCESS1-3
Reuse existing file: bilinear_145x192_192x288_peri.nc
CNRM-CM5
Reuse existing file: bilinear_128x256_192x288_peri.nc
IPSL-CM5B-LR
Reuse existing file: bilinear_96x96_192x288_peri.nc
GFDL-ESM2G
Reuse existing file: bilinear_90x144_192x288_peri.nc
MIROC-ESM
Reuse existing file: bilinear_64x128_192x288_peri.nc
FGOALS-g2
Reuse existing file: bilinear_60x128_192x288_peri.nc
bcc-csm1-1
Reuse existing file: bilinear_64x128_192x288_peri.nc
BNU-ESM
Reuse existing file: bilinear_64x128_192x288_peri.nc
CanESM2
Reuse existing file: bilinear_64x128_192x288_peri.nc
CCSM4
Reuse existing file: bilinear_192x288_192x288_peri.nc
CSIRO-Mk3-6-0
Reuse existing file: bilinear_96x192_192x288_peri.nc
FGOALS-s2
Reuse existing file: bilinear_108x128_192x288_peri.nc
GFDL-CM3
Reuse existing file: bilinear_90x144_192x288_peri.nc
inmcm4
Reuse existing file: bilinear_120x180_192x288_peri.nc
IPSL-CM5A-LR
Reuse existing file: bilinear_96x96_192x2

In [4]:
models = ['ACCESS1-0','ACCESS1-3','CNRM-CM5','IPSL-CM5B-LR', 'GFDL-ESM2G',
         'MIROC-ESM', 'FGOALS-g2','bcc-csm1-1','BNU-ESM','CanESM2','CCSM4',
          'CSIRO-Mk3-6-0','FGOALS-s2','GFDL-CM3','inmcm4',
         'IPSL-CM5A-LR','MIROC5','MPI-ESM-LR','MPI-ESM-P','MRI-CGCM3','NorESM1-M']

# start by reading in the air temperature kernel
ta_kern = xr.open_dataset('/dx05/tylerj/d10/Arctic_Research/CMIP5_Arctic_Amplification/\
CAM5_kernels/t.kernel.plev.nc',use_cftime=True)
lats = ta_kern.lat
lons = ta_kern.lon
# some models have higher vertical resolution than the kernels. We only need 17 levels
plevs = [float(p) for p in ta_kern.plev]

# create a dataset with lat/lons for regridding
ds_out = xr.Dataset({'lat': (['lat'], lats),jj
                     'lon': (['lon'], lons),
                    }
                   )

for mod in models:
    print(mod)
    # rlus for BNU-ESM is cut off, so we will use the last 50 years offset by 1 year
    if(mod=='BNU-ESM'):
        start = -612
        end = -12
    else:
        start = -600
        end = None
    # read in first 150 years of surface pressure file
    ps = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/ps_Amon_'+mod+'_*.nc',
                          parallel=True,
                           combine='by_coords',use_cftime=True).ps.isel(time=slice(start,end))

    # produce a regridder to map input to CAM5 horizontal grid
    regridder = xe.Regridder(ps,ds_out,'bilinear',periodic=True,reuse_weights=True)

    # regrid the surface pressure, chunk it
    ps = regridder(ps.compute())
    lat = ps.lat
    lon = ps.lon

    # we require the pressure level axis from a vertically varying variable, such as air temp
    ta = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/ta_Amon_'+mod+'_*.nc',
                          parallel=True,combine='by_coords',
                          use_cftime=True).ta.isel(time=0).sel(plev=plevs)
    plev_1D = ta.plev
    plev_da,foo = xr.broadcast(plev_1D,ps)

    # make bounds array to compute thickness later
    bounds_array = [96250., 88750., 77500., 65000., 55000., 45000., 35000., 27500.,
                        22500., 17500., 12500.,  8500.,  6000.,  4000.,  2500.,  1500.,0]
    bounds_da,foo = xr.broadcast(xr.DataArray(bounds_array,coords={'plev':plev_da.plev},dims='plev'),ps)

    ps = np.array(ps)
    dz = np.zeros_like(plev_da)

    # let's specify some dimensions:
    # ps is 3D (time,lat,lon)
    # we will be iterating through the vertical direction

    for i in range(17):
        # here is where we have to be very space conscious
        # dask does not allow item assignment as it is difficult to parallelize
        plev = np.array(plev_da.isel(plev=i))
        if(i==0):
            # grab bounds and plev, make arrays
            bounds = np.array(bounds_da.isel(plev=i))
            # set the lower layer thickness to be the surface pressure minus the lower bound
            dz[i,:,:,:] = ps[:,:,:] - bounds[:,:,:]
            # if the surface pressure is less than lowest pressure level, set to 0
            dz[i,plev[:,:,:] > ps[:,:,:]] = 0
        else:
            bounds = np.array(bounds_da.isel(plev=slice(i-1,i+1)))
            # where the lower bound is below the surface,
            # set thickness to be difference between upper bound and surface pressure
            # otherwise, do nothing
            dz[i,:,:,:] = np.where(bounds[0,:,:,:] > ps[:,:,:],
                                   ps[:,:,:] - bounds[1,:,:,:],
                                   dz[i,:,:,:])
            # where the center pressure is below the surface, set thickness to 0
            dz[i,plev[:,:,:] > ps[:,:,:]] = 0

    # convert dz to dataarray, save it out
    dz_da = xr.DataArray(dz,coords={'plev':plev_da.plev,'time':plev_da.time,'lat':plev_da.lat,
                                    'lon':plev_da.lon},
                             dims=['plev','time','lat','lon']).rename('dz').transpose('time',
                                                                                      'plev','lat','lon')

    del dz
    
    dz_da.to_netcdf('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/' + mod + '_dz_strat_pic.nc')

ACCESS1-0
Reuse existing file: bilinear_145x192_192x288_peri.nc
ACCESS1-3
Reuse existing file: bilinear_145x192_192x288_peri.nc
CNRM-CM5
Reuse existing file: bilinear_128x256_192x288_peri.nc
IPSL-CM5B-LR
Reuse existing file: bilinear_96x96_192x288_peri.nc
GFDL-ESM2G
Reuse existing file: bilinear_90x144_192x288_peri.nc
MIROC-ESM
Reuse existing file: bilinear_64x128_192x288_peri.nc
FGOALS-g2
Reuse existing file: bilinear_60x128_192x288_peri.nc
bcc-csm1-1
Reuse existing file: bilinear_64x128_192x288_peri.nc
BNU-ESM
Reuse existing file: bilinear_64x128_192x288_peri.nc
CanESM2
Reuse existing file: bilinear_64x128_192x288_peri.nc
CCSM4
Reuse existing file: bilinear_192x288_192x288_peri.nc
CSIRO-Mk3-6-0
Reuse existing file: bilinear_96x192_192x288_peri.nc
FGOALS-s2
Reuse existing file: bilinear_108x128_192x288_peri.nc
GFDL-CM3
Reuse existing file: bilinear_90x144_192x288_peri.nc
inmcm4
Reuse existing file: bilinear_120x180_192x288_peri.nc
IPSL-CM5A-LR
Reuse existing file: bilinear_96x96_192x2